# Загрузка Pandas и очистка данных

In [1483]:
import pandas as pd
import ast
import re

In [1484]:
df = pd.read_csv('main_task.csv')

## Анализ источника данных

In [1485]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      40000 non-null  object 
 1   City               40000 non-null  object 
 2   Cuisine Style      30717 non-null  object 
 3   Ranking            40000 non-null  float64
 4   Rating             40000 non-null  float64
 5   Price Range        26114 non-null  object 
 6   Number of Reviews  37457 non-null  float64
 7   Reviews            40000 non-null  object 
 8   URL_TA             40000 non-null  object 
 9   ID_TA              40000 non-null  object 
dtypes: float64(3), object(7)
memory usage: 3.1+ MB


## Предобработка

#### Вспомогательные функции

In [1486]:
col_city = 'City'
col_cuisines = 'Cuisine Style'
col_rest_id = 'Restaurant_id'
col_price = 'Price Range'
col_rating = 'Rating'
col_reviews_len = 'Number of Reviews'
col_reviews = 'Reviews'
col_city_reviews = 'Mean City Reviews'
col_review_days = 'Days Between Reviews'
col_last_review = 'Last Review Days'
col_positive = 'Positive Reviews'

date_pattern = re.compile('\d+/\d+/\d+')

review_keywords = ['good', 'delicious', 'best', 'gem', 'excellent', 'amazing', 'great', 'awesome', 'terrible', 'bad', 'worst']

today = pd.to_datetime('today').date()
min_date = pd.to_datetime('1900-01-01').date()

cities = df[col_city].value_counts().index.values
mean_city_reviews = {}

for city in cities:
    mean_city_reviews[city] = round(df.query('City == @city')['Number of Reviews'].mean())


def days_since_last_review(reviews):
    """Считает количество дней, прошедших со времени последнего отзыва"""

    review_dates = date_pattern.findall(reviews)
    last_review_date = min_date

    for review_date in review_dates:
        date = pd.to_datetime(review_date).date()

        if date > last_review_date:
            last_review_date = date

    return (today - last_review_date).days


def days_between_reviews(reviews):
    """Считает количество дней между отзывами"""

    review_dates = date_pattern.findall(reviews)

    if len(review_dates) == 2:
        date1 = pd.to_datetime(review_dates[0]).date()
        date2 = pd.to_datetime(review_dates[1]).date()

        return abs((date1 - date2).days)

    return 0


def fill_cuisines(cuisines):
    """Преобразует строку, содержащую список в объект списка"""

    return ['Unknown'] if pd.isna(cuisines) else ast.literal_eval(cuisines)


def parse_review(review, keyword):
    """Проверяет есть ли в отзыве слова, имеющие положительное значение"""

    # for keyword in positive_words:
    return 1 if keyword in review else 0

#### Преобразуем формат данных в Series и создаем дополнительные характеристики

In [1487]:
df[col_cuisines] = df.apply(lambda row: fill_cuisines(row[col_cuisines]), axis=1)
df[col_review_days] = df.apply(lambda row: days_between_reviews(row[col_reviews]), axis=1)
df[col_last_review] = df.apply(lambda row: days_since_last_review(row[col_reviews]), axis=1)
df[col_city_reviews] = df.apply(lambda row: mean_city_reviews[row[col_city]], axis=1)

# df[col_positive] = df.apply(lambda row: parse_review(row[col_reviews]), axis=1)

## Очистка данных

#### Заполняем пропуски

In [1488]:
df[col_reviews_len] = df[col_reviews_len].fillna(df[col_reviews_len].mean())
df[col_price] = df[col_price].fillna(df[col_price].value_counts().index[0])

## Feature Engineering

#### Создаем dummy-признаки для ценовой категории

In [1489]:
price_ranges = pd.get_dummies(df[col_price])
price_ranges.columns = ['Inexpensive', 'Middle Price', 'Expensive']

for c in price_ranges.columns:
    df[c] = price_ranges[c].values


#### Создаем dummy-признак количества кухонь

In [1490]:
df[col_cuisines] = df.apply(lambda row: len(row[col_cuisines]), axis=1)

#### Создаем dummy-признаки по городам

In [1491]:
city_ranges = pd.get_dummies(df[col_city])

for c in city_ranges.columns:
    df[c] = city_ranges[c].values

#### Создаем dummy-признаки по ключевым словам в отзывах

In [1492]:
for keyword in review_keywords:
    df[keyword] = df.apply(lambda row: parse_review(row[col_reviews], keyword), axis=1)

#### Удаляем столбцы с dtype Object

In [1493]:
object_columns = []

for c in df.columns:
    if(df[c].dtype == 'object' and c != col_rest_id):
        object_columns.append(c)

df.drop(object_columns, axis=1, inplace=True)

# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [1494]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = df.drop([col_rest_id, col_rating], axis = 1)
y = df[col_rating]

In [1495]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [1496]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Создаём, обучаем и тестируем модель

In [1497]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [1498]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [1499]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.21238199999999996
